In [1]:
#Save a model
import numpy as np

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
# more info on callbakcs: https://keras.io/callbacks/ model saver is cool too.
from tensorflow.keras.callbacks import TensorBoard
import pickle
import time

pickle_in = open("X.picle","rb")
X = pickle.load(pickle_in)

pickle_in = open("y.picle","rb")
y = pickle.load(pickle_in)
y=np.array(y)
X = X/255.0

dense_layers = [0]
layer_sizes = [64]
conv_layers = [3]

for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            NAME = "{}-conv-{}-nodes-{}-dense-{}".format(conv_layer, layer_size, dense_layer, int(time.time()))
            print(NAME)

            model = Sequential()

            model.add(Conv2D(layer_size, (3, 3), input_shape=X.shape[1:]))
            model.add(Activation('relu'))
            model.add(MaxPooling2D(pool_size=(2, 2)))

            for l in range(conv_layer-1):
                model.add(Conv2D(layer_size, (3, 3)))
                model.add(Activation('relu'))
                model.add(MaxPooling2D(pool_size=(2, 2)))

            model.add(Flatten())

            for _ in range(dense_layer):
                model.add(Dense(layer_size))
                model.add(Activation('relu'))

            model.add(Dense(1))
            model.add(Activation('sigmoid'))

            tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))

            model.compile(loss='binary_crossentropy',
                          optimizer='adam',
                          metrics=['accuracy'],
                          )

            model.fit(X, y,
                      batch_size=32,
                      epochs=10,
                      validation_split=0.3,
                      callbacks=[tensorboard])

model.save('64x3-CNN.model')

3-conv-64-nodes-0-dense-1594054543
Epoch 1/10
546/546 [==============================] - 24s 44ms/step - loss: 0.6522 - accuracy: 0.6082 - val_loss: 0.5968 - val_accuracy: 0.6871
Epoch 2/10
546/546 [==============================] - 22s 41ms/step - loss: 0.5524 - accuracy: 0.7192 - val_loss: 0.5019 - val_accuracy: 0.7626
Epoch 3/10
546/546 [==============================] - 20s 37ms/step - loss: 0.4916 - accuracy: 0.7623 - val_loss: 0.4769 - val_accuracy: 0.7719
Epoch 4/10
546/546 [==============================] - 24s 45ms/step - loss: 0.4550 - accuracy: 0.7859 - val_loss: 0.4648 - val_accuracy: 0.7854
Epoch 5/10
546/546 [==============================] - 24s 44ms/step - loss: 0.4203 - accuracy: 0.8106 - val_loss: 0.4443 - val_accuracy: 0.7968
Epoch 6/10
546/546 [==============================] - 21s 39ms/step - loss: 0.3865 - accuracy: 0.8267 - val_loss: 0.4347 - val_accuracy: 0.7973
Epoch 7/10
546/546 [==============================] - 22s 40ms/step - loss: 0.3592 - accuracy: 0.8399

In [5]:
#Make prediction
import cv2
import tensorflow as tf

CATEGORIES = ["Dog", "Cat"]


def prepare(filepath):
    IMG_SIZE = 50  # 50 in txt-based
    img_array = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
    new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
    return new_array.reshape(-1, IMG_SIZE, IMG_SIZE, 1)


model = tf.keras.models.load_model("64x3-CNN.model")

prediction = model.predict([prepare('CAT.jpg')])
print(prediction)  # will be a list in a list.
print(CATEGORIES[int(prediction[0][0])])

[[0.]]
Dog
